In [ ]:
    # This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from scipy.stats import probplot, pearsonr
import matplotlib.pyplot as plt

import seaborn as sns
import pytorch_lightning as pl
import torch
import torch.nn as nn
from torch.nn import Linear 

import pickle
from torch.autograd import Variable
import torch.utils.data as Data
import math
import time
import datetime
import os
import random


# **Saving 'train_data' to 'pkl', for less time to load 'train_data'**

In [ ]:
'''
train_dtypes = {f'f_{i}': np.float32 for i in range(300)}
train_dtypes['investment_id'] = np.uint16
train_dtypes['time_id'] = np.uint16
train_dtypes['target'] = np.float32

df_train = pd.read_csv('../input/ubiquant-market-prediction/train.csv', usecols=list(train_dtypes.keys()), dtype=train_dtypes)
print(f'Training Set Shape: {df_train.shape} - Memory Usage: {df_train.memory_usage().sum() / 1024 ** 2:.2f} MB')
df_train.to_pickle('train.pkl')
'''

# **load data from .pkl**

# **some data analysis**

# **data loaders**

# **transformers base model**

In [ ]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
from torch.nn import Linear 



def gen_trg_mask(length, device):
    mask = torch.tril(torch.ones(length, length, device=device)) == 1

    mask = (
        mask.float()
        .masked_fill(mask == 0, float("-inf"))
        .masked_fill(mask == 1, float(0.0))
    )

    return mask

class Transformers(pl.LightningModule):
    def __init__(
        self,
        n_encoder_inputs,
        n_decoder_inputs,
        channels=64,
        dropout=0.1,
        in_ebddim = 128,
        tar_ebddim = 128,
        ):
        super(Transformers, self).__init__() 

        self.dropout = dropout

        self.input_pos_embedding = torch.nn.Embedding(in_ebddim, embedding_dim=channels)
        self.target_pos_embedding = torch.nn.Embedding(tar_ebddim, embedding_dim=channels)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=channels,
            nhead=8,
            dropout=self.dropout,
            dim_feedforward=4 * channels,
        )
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=channels,
            nhead=8,
            dropout=self.dropout,
            dim_feedforward=4 * channels,
        )

        self.encoder = torch.nn.TransformerEncoder(encoder_layer, num_layers=8)
        self.decoder = torch.nn.TransformerDecoder(decoder_layer, num_layers=8)

        self.input_projection = Linear(n_encoder_inputs, channels)
        self.output_projection = Linear(n_decoder_inputs, channels)

        self.linear = Linear(channels, 1)

        self.do = nn.Dropout(p=self.dropout)

    def encode_src(self, src):
        # src = src.long()
        src_start = self.input_projection(src).permute(1, 0, 2)

        in_sequence_len, batch_size = src_start.size(0), src_start.size(1)
        pos_encoder = (
            torch.arange(0, in_sequence_len, device=src.device)
            .unsqueeze(0)
            .repeat(batch_size, 1)
        )

        pos_encoder = self.input_pos_embedding(pos_encoder).permute(1, 0, 2)

        src = src_start + pos_encoder

        src = self.encoder(src) + src_start

        return src

    def decode_trg(self, trg, memory):
        # trg = trg.long()
        trg_start = self.output_projection(trg).permute(1, 0, 2)

        out_sequence_len, batch_size = trg_start.size(0), trg_start.size(1)

        pos_decoder = (
            torch.arange(0, out_sequence_len, device=trg.device)
            .unsqueeze(0)
            .repeat(batch_size, 1)
        )
        pos_decoder = self.target_pos_embedding(pos_decoder).permute(1, 0, 2)

        trg = pos_decoder + trg_start

        trg_mask = gen_trg_mask(out_sequence_len, trg.device)

        out = self.decoder(tgt=trg, memory=memory, tgt_mask=trg_mask) + trg_start

        out = out.permute(1, 0, 2)

        out = self.linear(out)

        return out

    def forward(self, x):
        src, trg = x

        src = self.encode_src(src)

        out = self.decode_trg(trg=trg, memory=src)

        return out
    
    

n_classes = 100

source = torch.rand(size=(32, 16, 9))
target_in = torch.rand(size=(32, 16, 9))
target_out = torch.rand(size=(32, 16, 1))
ts = Transformers(n_encoder_inputs=9, n_decoder_inputs=9)

pred = ts((source, target_in))


# **train models**

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Feb 17 17:24:09 2022

@author: jaycezhao
"""

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from scipy.stats import probplot, pearsonr
import matplotlib.pyplot as plt

import seaborn as sns
import pytorch_lightning as pl
import torch
import torch.nn as nn
from torch.nn import Linear 

import pickle
from torch.autograd import Variable
import torch.utils.data as Data
import math
import time
import datetime
import os
import random

df_train = pd.read_pickle('../input/ubiquant-train-pkl/train.pkl')

cols = list(df_train.columns)
fearures_col = cols[3:]
keys = df_train[['investment_id','time_id']].groupby(by=['investment_id','time_id']).first().reset_index()

investment_ids = list(keys['investment_id'].unique())


def data_loaders(df_train_raw,investment_ids,fearures_col):
    df_train = df_train_raw.copy()
    target = []
    train_x = []
    train_y = []
    
    target_test = []
    train_x_test = []
    train_y_test = []
    train_flag = False
    test_flag = False
    for ids in range(len(investment_ids[:])):
    #investment_id = investment_ids[0]
        investment_id = investment_ids[ids]
        # print('\r make data_loaders****totall:{totall}/now:{ids}/investment_id:{investment_id}'.format(totall=len(investment_ids[:]),ids=ids+1,investment_id=investment_id),end='')
        data_temp = df_train[df_train['investment_id']==investment_id]   
        data_temp_train = data_temp[(data_temp['time_id']<1066)&(data_temp['time_id']>700)]
        if len(data_temp_train)>0:
            train_flag = True
            target.extend(data_temp_train['target'].tolist()[:])
            train_x.extend(data_temp_train[fearures_col].values.tolist()[:])
            #embadding transform
            temp_y = data_temp_train[fearures_col[::-1]].values.tolist()[:]
            train_y.extend(temp_y)
        data_temp_test = data_temp[data_temp['time_id']>1066]
        if len(data_temp_test)>0:
            test_flag = True
            target_test.extend(data_temp_test['target'].tolist()[:])
            train_x_test.extend(data_temp_test[fearures_col].values.tolist()[:])
            #embadding transform
            temp_test_y = data_temp_test[fearures_col[::-1]].values.tolist()[:]
            train_y_test.extend(temp_test_y)          
    train_dataset = [train_x,train_y,target] 
    test_dataset = [train_x_test,train_y_test,target_test]
    return train_dataset,test_dataset,train_flag,test_flag
train_dataset,test_dataset,train_flag,test_flag = data_loaders(df_train,investment_ids[:5],fearures_col)    

def gen_trg_mask(length, device):
    mask = torch.tril(torch.ones(length, length, device=device)) == 1

    mask = (
        mask.float()
        .masked_fill(mask == 0, float("-inf"))
        .masked_fill(mask == 1, float(0.0))
    )

    return mask

class Transformers(pl.LightningModule):
    def __init__(
        self,
        n_encoder_inputs,
        n_decoder_inputs,
        channels=64,
        dropout=0.1,
        in_ebddim = 128,
        tar_ebddim = 128,
        ):
        super(Transformers, self).__init__() 

        self.dropout = dropout

        self.input_pos_embedding = torch.nn.Embedding(in_ebddim, embedding_dim=channels)
        self.target_pos_embedding = torch.nn.Embedding(tar_ebddim, embedding_dim=channels)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=channels,
            nhead=8,
            dropout=self.dropout,
            dim_feedforward=4 * channels,
        )
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=channels,
            nhead=8,
            dropout=self.dropout,
            dim_feedforward=4 * channels,
        )

        self.encoder = torch.nn.TransformerEncoder(encoder_layer, num_layers=8)
        self.decoder = torch.nn.TransformerDecoder(decoder_layer, num_layers=8)

        self.input_projection = Linear(n_encoder_inputs, channels)
        self.output_projection = Linear(n_decoder_inputs, channels)

        self.linear = Linear(channels, 1)

        self.do = nn.Dropout(p=self.dropout)

    def encode_src(self, src):
        # src = src.long()
        src_start = self.input_projection(src).permute(1, 0, 2)

        in_sequence_len, batch_size = src_start.size(0), src_start.size(1)
        pos_encoder = (
            torch.arange(0, in_sequence_len, device=src.device)
            .unsqueeze(0)
            .repeat(batch_size, 1)
        )

        pos_encoder = self.input_pos_embedding(pos_encoder).permute(1, 0, 2)

        src = src_start + pos_encoder

        src = self.encoder(src) + src_start

        return src

    def decode_trg(self, trg, memory):
        # trg = trg.long()
        trg_start = self.output_projection(trg).permute(1, 0, 2)

        out_sequence_len, batch_size = trg_start.size(0), trg_start.size(1)

        pos_decoder = (
            torch.arange(0, out_sequence_len, device=trg.device)
            .unsqueeze(0)
            .repeat(batch_size, 1)
        )
        pos_decoder = self.target_pos_embedding(pos_decoder).permute(1, 0, 2)

        trg = pos_decoder + trg_start

        trg_mask = gen_trg_mask(out_sequence_len, trg.device)

        out = self.decoder(tgt=trg, memory=memory, tgt_mask=trg_mask) + trg_start

        out = out.permute(1, 0, 2)

        out = self.linear(out)

        return out

    def forward(self, x):
        src, trg = x

        src = self.encode_src(src)

        out = self.decode_trg(trg=trg, memory=src)

        return out
    

source = torch.rand(size=(32, 16, 9))
target_in = torch.rand(size=(32, 16, 9))
target_out = torch.rand(size=(32, 16, 1))
ts = Transformers(n_encoder_inputs=9, n_decoder_inputs=9)

pred = ts((source, target_in))



def set_seed(seed):
    os.environ['PYTHONHASHSEED']=str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    
class DataSet_Ids(Data.Dataset):
    def __init__(self, dataset_ids):
        super(DataSet_Ids, self).__init__()
        self.dataset_ids = dataset_ids
    def __len__(self):
        return np.shape(self.dataset_ids)[0]
    def __getitem__(self, idx):
        return self.dataset_ids[idx]


def make_dateloader_iters(train_x_raw,train_y_raw,target_raw):
    steps_all = int((len(target_raw)-windows)/step)
    train_xs = []
    train_ys = []
    targets = []
    for iters in range(steps_all):
        train_x = train_x_raw[iters*step:iters*step+windows]
        train_y = train_y_raw[iters*step:iters*step+windows]
        target = target_raw[iters*step:iters*step+windows]                
        
        x = np.array(train_x)

        y = np.array(train_y)

        z = np.expand_dims(np.array(target),1)
               
        
        # x = np.array([np.array(train_x),np.array(train_y),np.array(train_x)])
        # x = np.swapaxes(x,0,2)
        # x = np.swapaxes(x,0,1)
        # y = np.array([np.array(train_y),np.array(train_x),np.array(train_y)])
        # y = np.swapaxes(y,0,2)
        # y = np.swapaxes(y,0,1)
        # z = np.expand_dims(np.array(target),1)
        # z = z[:,:,np.newaxis]        
        
        train_xs.append(x)
        train_ys.append(y)
        targets.append(z)
    return train_xs,train_ys,targets

class Train_DataSet(Data.Dataset):
    def __init__(self, enc_inputs, dec_inputs, dec_outputs):
        super(Train_DataSet, self).__init__()
        self.enc_inputs = enc_inputs
        self.dec_inputs = dec_inputs
        self.dec_outputs = dec_outputs
  
    def __len__(self):
        return np.shape(self.enc_inputs)[0]
    
    def __getitem__(self, idx):
        return self.enc_inputs[idx], self.dec_inputs[idx], self.dec_outputs[idx]




In [ ]:
len(investment_ids)

In [ ]:
df_train


In [ ]:
epoch = 32
batch_size = 128
step = 32
windows = 32
SEED = 42
LR=1e-5
set_seed(SEED)
timest = time.time()


n_decoder_inputs=len(fearures_col)
n_encoder_inputs=len(fearures_col)
# print(n_decoder_inputs,n_encoder_inputs)
investment_ids = list(keys['investment_id'].unique())

model = Transformers(n_encoder_inputs=n_encoder_inputs, 
                     n_decoder_inputs=n_decoder_inputs,
                     channels=64,in_ebddim = 128,tar_ebddim = 128)
los_min = 1000
gpus = [0]
cuda_gpu = torch.cuda.is_available()
if (cuda_gpu):                   
    model = torch.nn.DataParallel(model,device_ids=gpus).cuda()  
for h in range(epoch): 
    model.train()
    # lr = float(LR*(1*math.cos((0.5*math.pi*h/epoch))))
    lr = float(LR*(1-(h/epoch)**0.5))
    optimizer = torch.optim.Adam(model.parameters(),lr)
    loss_list = []
    loss_evl_list = [2]    
    for steps, investment_id in enumerate(investment_ids[::8]):
        time_now = datetime.datetime.now().strftime('%Y_%m_%d ## %H:%M:%S')
        train_dataset,test_dataset,train_flag,test_flag = data_loaders(df_train,[investment_id],fearures_col)
        #print('Training ############# investment_ids:{investment_id}'.format(investment_id=investment_id))
        if train_flag:
            [train_x_raw,train_y_raw,target_raw] = train_dataset 
            train_xs,train_ys,targets = make_dateloader_iters(train_x_raw,train_y_raw,target_raw)
            
            train_xs,train_ys,targets = torch.FloatTensor(train_xs), torch.FloatTensor(train_ys), torch.FloatTensor(targets)
            train_loader = Data.DataLoader(Train_DataSet(train_xs,train_ys,targets),batch_size=16, shuffle=False,num_workers=0)
            for iters,(x,y,z) in enumerate(train_loader):
                # print(np.shape(x),np.shape(y),np.shape(z))
            
                # x,y,z = torch.FloatTensor(x),torch.FloatTensor(y),torch.FloatTensor(z)

                if (cuda_gpu):
                    encin_tr, decin_tr, decout_tr = Variable(x).cuda(),Variable(y).cuda(),Variable(z).cuda()
                else:           
                    encin_tr, decin_tr, decout_tr = Variable(x),Variable(y),Variable(z)
                # print('\nxz',encin_tr.size(),decin_tr.size(),decout_tr.size())   
                output = model((encin_tr, decin_tr))
                # print(output.size())
                loss_fun = nn.MSELoss()
                loss = loss_fun(decout_tr,output)
    
                loss_list.append(loss.cpu().detach().numpy()) 
                #清空上一次梯度
                optimizer.zero_grad()
                #误差反向传递
                loss.backward()
                #优化器参数更新
                optimizer.step()
            if steps%5==0:    
                print('\r Train steps average loss is:{}/ Time is: {} /totall:{epoch}/now:{h} / investment_id is {investment_id}'.format(np.mean(loss_list),time_now,epoch=epoch,h=h+1,investment_id=investment_id),end='')

    for steps, investment_id in enumerate(investment_ids[::8]):
        time_now = datetime.datetime.now().strftime('%Y_%m_%d ## %H:%M:%S')
        if test_flag:
            [test_x_raw,test_y_raw,target_test] = test_dataset 
            test_xs,test_ys,target_tests = make_dateloader_iters(test_x_raw,test_y_raw,target_test)
            
            test_xs,test_ys,target_tests = torch.FloatTensor(test_xs), torch.FloatTensor(test_ys), torch.FloatTensor(target_tests)
            test_loader = Data.DataLoader(Train_DataSet(test_xs,test_ys,target_tests),batch_size=16, shuffle=False,num_workers=0)
            for iters,(x,y,z) in enumerate(test_loader):
                # print(np.shape(x),np.shape(y),np.shape(z))
            
                # x,y,z = torch.FloatTensor(x),torch.FloatTensor(y),torch.FloatTensor(z)
        
                if (cuda_gpu):
                    encin_te, decin_te, decout_te = Variable(x).cuda(),Variable(y).cuda(),Variable(z).cuda()
                else:           
                    encin_te, decin_te, decout_te = Variable(x),Variable(y),Variable(z)
                # print('\nxz',encin_tr.size(),decin_tr.size(),decout_tr.size())   
                output_te = model((encin_te, decin_te))
                # print(output.size())
                loss_fun = nn.MSELoss()
                loss_evl = loss_fun(decout_te,output_te)
                loss_evl_list.append(loss_evl.cpu().detach().numpy()) 
            if steps%5==0:
                print('\r Evl steps average loss is:{}, Time is: {} totall:{epoch}/now:{h} / investment_id is {investment_id}'.format(np.mean(loss_evl_list),time_now,epoch=epoch,h=h+1,investment_id=investment_id),end='')
        losses_avev = np.mean(loss_evl_list)
        if losses_avev <= los_min:
            los_min = losses_avev
            epo = h+1
            print('Save models '+"./model_"+str(h+1)+".pth\n")
            torch.save(model.state_dict(),"model.pth")  